In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from hybrid_search import hybrid_search

In [3]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
voyage_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'


qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = ElasticsearchBM25()

Connecting to Elasticsearch at elasticsearch:9200


In [31]:
q = """
enclose thinking into <thinking> tag and answer into <answer> tag. 

While thinking extract all entities like people, companies, products, ideas, action points, tasks as simple list.
then generate consise summary of the meeting based on the extracted entities.
Highlight entities in bold markdown
"""

In [52]:
q = """
generate consise summary of the meeting based on the extracted entities.
Highlight entities in bold markdown
"""

In [53]:
meeting_ids = ['4f64ba1e-fd80-4b5d-a1be-345225028f6c']

In [54]:
r = await hybrid_search(q, qdrant_engine, es_engine,meeting_ids=meeting_ids)

In [55]:
import pandas as pd
df = pd.DataFrame(r['results']).sort_values('meeting_id', ascending=False)

In [56]:
def format_meeting_context(df):
    # Create meeting_id to integer mapping
    unique_meetings = df['meeting_id'].unique()
    meeting_map = {mid: idx+1 for idx, mid in enumerate(unique_meetings)}
    meeting_map_reverse = {v:k for k,v in meeting_map.items()}
    
    # Sort by timestamp and meeting_id
    df = df.sort_values(['timestamp', 'meeting_id'])
    
    meetings = []
    for meeting_id, group in df.groupby('meeting_id'):
        # Get integer meeting id
        int_meeting_id = meeting_map[meeting_id]
        
        # Format timestamp
        timestamp = pd.to_datetime(group['timestamp'].iloc[0])
        date_header = f"## Meeting {int_meeting_id} - {timestamp.strftime('%B %d, %Y %H:%M')}"
        
        # Format topic if available
        topic = group['topic'].iloc[0]
        if topic:
            date_header += f"\nTopic: {topic}"
            
        # Collect content with formatted time
        content_items = []
        for _, row in group.iterrows():
            time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
            content_line = f"- {time_prefix} {row['content']}"
            content_items.append(content_line)
        
        content = "\n".join(content_items)
        meetings.append(f"{date_header}\n\n{content}\n")
    
    return "\n".join(meetings), meeting_map_reverse

# Create formatted context and store mapping
input_context, meeting_id_map = format_meeting_context(df)

In [57]:
meeting_id_map

{1: '4f64ba1e-fd80-4b5d-a1be-345225028f6c'}

In [58]:
from core import generic_call,system_msg,user_msg,count_tokens

In [59]:
from IPython.display import display, Markdown
display(Markdown(input_context))


## Meeting 1 - December 11, 2024 18:31
Topic: Action Points and Next Steps

- [41:10] Dmitriy Grankin:  the audience how we would describe it and combine a set of or at least basically we can just can combine this list together and um and and start outreaching so that eventually uh i uh yeah i want to talk to these guys so because talking to these guys is already something that is that would bring value yeah yeah so the idea is the final product is is a bit earlier not not the final product but a valuable product valuable product is a meeting with this guy okay yeah just because we he understand this guy's not just content creators i i suspect and suppose i Maybe a bunch of them would be doing content creation as their main thing, as their main business.  We have meetings with the people who are creating content for the sake of generating leads.  their business yeah okay so this way this will not be like spherical content creators But these guys would be domain experts.  Some guys that are doing very specific in a very specific location for a very specific set of customers.  okay so in terms of action points what can what should we leave yeah um  Okay, to wrap up, we want to have meetings with those, we want to create a set of short descriptions about who we are looking for, these short descriptions.  would not be describing the influencers themselves I would be describing their audience okay so a few descriptions of the audience they are uh they are targeting and starting from that from these descriptions you can you can do like uh you can go and search for influencers already but if you use using my tool then the next step is creating a bunch of youtube queries specific queries um these guys are likely to make and then process this curious into a set of uh channels on youtube  yeah so basically
- [06:01] This chunk is part of a conversation between Dmitriy Grankin and Lara Vargas, where Dmitriy discusses strategies for identifying and engaging with micro-influencers in specific industries. He emphasizes the importance of targeting domain insiders who have a precise user base and can provide insights into potential customer profiles, as he seeks to refine his understanding of his ideal customer for a product or service.
Dmitriy Grankin:  deal customer for now and the idea is true yeah to talk to the it's an easy at least some kind of very specific way pipeline of getting in touch with with domain insiders  Okay. Sorry, with which insiders?  Domain means those that are doing something specific thing that they're i don't know maybe they are doing for equipment some kind of specific or they are in pharma or they are setting up some Jira for a company's location.  in in the UK for example right or in Brazil so they are very local they are very specific they have a very precise user base a client base so they are very local about what they are doing and about where they are doing what they are doing  So the idea is to penetrate through these people who are already in contact with big companies we don't know what what  companies they are servicing okay we don't know where there is a big companies or smaller companies or whatever but if they are having something to say about this thing right on youtube they're probably having some idea ideas about how they're going to convert this views into money, right?  So most likely they are, they have some, uh, local business, I mean local, not just in terms of geography, but local in terms of very specific, you know, very specific user base.  it's a very specific customer they're there most likely having their own idea customer profile they all And since I don't know my ideal customer profile, but I have a system which can service a variety of business use cases.  This way I want to find an ideal customer profile or a bunch of those.  Okay.  Okay, and what do you have on your mind about that?  Do you want to participate in this thing or you have your own ideas?
- [09:22] Lara Vargas:  Yeah, so from what I gathered from our conversation last week, the briefing was think about ways to think to make VEXA a useful tool outside the...  usual way of use, think about tips and tricks to use it, to make content around it, and make research and About what type of different uses that would be.  So I brought a few that I thought about.  And so the first one would be a training Feedback for communication, so for example, people that are coaches, career coaches, you know, that they can use VEXA for their clients.  when they are in coaching their clients really, and then they can use a transcription to analyze how the person communicates during the meetings, look for, patterns and their communication as, for example, if they use some kind of excessive terms or if they interrupt sometimes how efficient the message is that they are transcribing and then sorry okay okay yeah go ahead and then add the social media that i would focus for this specific The specific target would be LinkedIn and Instagram.  The strategy would be short videos, visual testimonials and carousel.
- [10:00] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing potential use cases for the VEXA tool. Lara proposes using VEXA for training feedback in communication, specifically targeting coaches and career coaches, and outlines a strategy for promoting this use case on social media platforms like LinkedIn and Instagram.
Lara Vargas:  And so the first one would be a training Feedback for communication, so for example, people that are coaches, career coaches, you know, that they can use VEXA for their clients.  when they are in coaching their clients really, and then they can use a transcription to analyze how the person communicates during the meetings, look for, patterns and their communication as, for example, if they use some kind of excessive terms or if they interrupt sometimes how efficient the message is that they are transcribing and then sorry okay okay yeah go ahead and then add the social media that i would focus for this specific The specific target would be LinkedIn and Instagram.  The strategy would be short videos, visual testimonials and carousel.
- [37:58] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing strategies for identifying and engaging with micro-influencers for marketing purposes. Lara shares her observations on influencer engagement rates, specifically highlighting the disparity between follower counts and actual viewer engagement during live sessions, emphasizing the importance of qualitative analysis in selecting influencers.
Lara Vargas:  So yeah, what I will do is create a log of possible influencers and look into that to see which one has a better potential between the what we are looking for of being a micro influencer and being able to deliver engagement you know so so for example Uma coisa que eu estava olhando, quando fizemos o Lucas e ele nos ofereceu fazer lives e tudo isso, It was, and we got like, I think, simultaneous.  I think it was six, but then afterwards he sent to me that it was about 15 people.  And I was seeing afterwards that there was so many Instagram accounts that I follow that some of them have 200 000 followers and they only get like six six uh viewers per life that they make so it was actually a person with a high engagement rate for the number of followers that he has, you know?  So this type of thing is something that a tool wouldn't be able to say, you know?  So, yeah, this is the type, yes.  Six is the number of people on a live...  watching him live or is a kind of what's the value of 6 and 15 you mentioned  it was six simultaneous people but it was a i think was 15 people overall so people coming and going in total it ended up being 15. just for for instagram i think and then i i have the stats i can send it all okay um yeah so so so  But starting from this, I would need to just think of it separately.
- [11:00] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing various use cases for the VEXA tool. Specifically, Lara is outlining the application of VEXA in qualitative research, highlighting its benefits for transcription and analysis of interviews conducted by specialized marketing companies.
Lara Vargas:  The second use case would be qualitative research.  So there is a few companies that do qualitative research So they have to make interviews and that is also a really good way of using VEXA because then they can have the transcription, and lots of times when you use a when you do qualitative research you make a recording that you have to listen to over and over again so vexa would do a very good way to to not have to go through the recording again you would just have to go through the transcription and then analyze it so a few quality  search you mean uh what what's the you you mean uh those guys who are listening to conversations by by other people yeah like yeah like uh like specialized marketing companies that's  specialized in research for other companies.
- [12:39] The chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing the potential applications of VEXA, a transcription tool, in various business contexts. Specifically, Lara is explaining how VEXA could be utilized in qualitative research, particularly in focus groups for product launches, highlighting its usefulness in analyzing participant feedback.
Lara Vargas:  So for example, if there is a big, a large company that wants to launch a product, and so they will make a focus group about that product you know yes so they will ask that focus group what they think about that product that normally happens with a record regarding that they make an interview with the person about what they think of the product.  So Vaxxer could be a really good use.  It's the kind of product
- [04:07] This chunk is part of a conversation between Dmitriy Grankin and Lara Vargas discussing strategies for influencer marketing, specifically focusing on engaging micro-influencers who have expertise in niche areas like project management tools (e.g., Jira). Dmitriy emphasizes the value of these influencers' audiences and their potential to source clients based on their domain expertise.
Dmitriy Grankin:  Yeah, and the perspective of these guys is also something that's a pretty valuable thing because they got their audience they've got their specific niche use case because if they are posting about setting up jira for whatever reason they are possibly sourcing sourcing clients from this so that's something about But their domain expertise is somewhere around that, so it's an interesting source of business use case.
- [17:12] This chunk is part of a conversation between Lara Vargas and Dmitriy Grankin discussing strategies for influencer marketing and content creation. Lara emphasizes the importance of engaging with influencers to understand their audience and the market before creating content, suggesting a pipeline approach to gather insights and develop targeted content based on those interactions.
Lara Vargas:  So if you want me, I can read the rest and then we can choose a few and i can make this pipeline  yeah okay so the pipeline is that i i don't think we need to create any content before we're actually well aware of what's being created and what's the audience about around this idea right so you if you would engage with those influencers first And then you will accommodate yourself in this problem, in this problem dimension, right?  You can basically very quickly turn around with different problem areas like this way, because today you can outreach say 20 manual speakers about this thing and set up a few meetings and this just a few meetings would give you a lot of information about if you really want to be do they want you to be what's their culture about that how actively they are looking for something new at all so that The pipeline is that you get these channels, you set up meetings, you get these meetings, and here is the product.  is already there because you had a few meetings with demand expertise in a very precise niche.  So it doesn't matter much about which problem area you are going to export this way right because you can just take a bunch of those randomly and start from those if they are just suddenly it's sound right right okay okay after you got this idea about what's going on, about this contact, about these people, this, uh, then you'd better engage with those creating content with those guys.  And just right after that, you're going to create your own content because you're already well aware of the problem, you have well data, the idea customer profile this way, possibly. And you have experience created content with those guys, you know, what works, what what does not. And After that, you're ready to create your own content.  That's how I feel about this.  Okay.
- [14:16] Dmitriy Grankin:  yeah um i like from the very beginning beginning to create pipelines that would make life much easier in terms of routines because outreaching someone you have a weak idea about LinkedIn is a pretty scary thing.  You just go to LinkedIn and you, you die there, right?  Because it's a social network and something.  going on there and you have a vague idea what you're what you're actually doing there so that's why uh this pipeline or other pipeline Like this, I mentioned. I just created a piece of code, which you drop in, like, how you put it.  this qualitative research specialist or you do it the other way. You would ask chat GPT about what is the likely that those guys would look on YouTube, Quantitative Research Specialist, right?  What would they be looking for on YouTube?  then you got a bunch of requests each of those you just drop in into into this piece of code to get nano-influencer's LinkedIn address.  linkedin or address or a name then you would then have to this specific name and go and check the content on linkedin and and outreach linkedin but at least at this point you are you have very specific things to do and if you have other pipelines like this uh so that you first you develop a system which is easy to develop and easy to to use actually yeah yeah otherwise it's um yeah it's uh just a matter of luck whether it will work this one it's Quantitative. Quantitative research to this, you know.  Yeah, okay. I actually brought other several ideas of audiences to target because from my understanding that was the idea like bring different ways to use So what I tried to do was bring in lots of different ways.  And so we would choose a few to focus on.
- [23:34] Dmitriy Grankin:  I was just piece of I'll share with you a piece of code Don't know if you can see much here. Yeah, I mean it's a code yeah this is the right code right so So here it is. So we've got we've got Search with channels how to stop Trello for product manager Management and here I'm going and to get I got the results and I got 10 videos with channels and description channel channel info title description and hopefully somewhere here we've got their contact information like they've got this Instagram gmail business contact that's what they shared in their in their channel Description, yeah Yeah, so maybe the easiest way would be just to first to come up with the list of like a list of user profiles and then to generate a list of their possible cures that they will be likely making on youtube and then uh parsing those channels that were created content recently with a not that huge amount of views and parse uh their contact information from those and that will be already like the list basically a list of contacts you are automatically getting a list of contacts so that the the manual part comes in the manual part is that you go and you outreach The next part is basically to set up a meeting.  To set up a meeting in order to talk to them. Basically, this is the result itself.
- [25:27] Lara Vargas:  I don't know if I understand this last part about the code, because do you want me to use to be using this code for the for the next steps as well uh using this code i don't think  it will work and i don't think it's possible uh think that maybe would go the other way is that we just I'll just put a bunch of contacts out of it okay yeah so tell me if it's so funny interested there's something about that you'll be interested to do it all because yeah  sorry what did what i don't don't understand yeah i'm tell me  if it's if you are interested at all about uh doing doing this thing  yeah i am i just i just don't understand uh what you mean because um You are, so you are showing me this piece of code and then through this piece of code you plan on getting the code.  contact information of the people on youtube so you can afterwards reach out to them to make videos with them but um the in terms of the the content that they promote.  That's what is in my mind.  I don't understand what type of content you want to focus on.  Do you think that we should just Por exemplo, a lista que eu tenho aqui. Talvez eu possa compartilhar com vocês também.  Oh, mas desculpe, está em português. Eu posso compartilhar com vocês mais tarde.  So I have a few different options.
- [05:01] This chunk is part of a conversation between Dmitriy Grankin and Lara Vargas discussing strategies for influencer marketing and content creation for Vexa. Dmitriy expresses the need to identify influencers who can effectively communicate Vexa's value to their audiences, highlighting his uncertainty about the ideal customer profile and the importance of engaging with domain experts.
Dmitriy Grankin:  But the idea is just to find an influencer who will be interested in explaining Vexa to their audience basically or even if not it's an interesting source of domain information.  It's like talking to these guys who bring a lot of generalized information from their specific niches they are in because like I am very I understand I have been very local about my ideas How to market this thing, how to position this thing, and who is my ideal customer?  I just basically have no idea who is my ideal.
- [35:22] Dmitriy Grankin:  okay how to what are the mechanics of of doing this thing what do you mean the mechanics the mechanics is something that you potentially can automate or least you know what to do specifically right you open up computer you yeah you create a new tab on your browser to type in this thing and you then go and search for this and that and then you the pipeline is should be something that one need to to rely on it on the chance it's to go from A to Z right so easy a a to Z it just straightforward way of going from a to z right it is like like if you're commuting from home to every day and it's not an adventure all the time yes so what I normally do  do in terms of this automatization is i will research by keywords in the social medias and then i will make a log of the potential um of the potential influencers that we can be targeting on and then i will be making uh engagement rates that part is all automatized and i will be comparing what the engagement rate is between all of them what what is the  engagement rate and how you assess it so the engagement rate depends on the  depends on a few things. Depends first on the type of social media that we are in.  It depends on, it can be a global engagement rate, it can be an engagement rate for the last month can be an engagement rate for the last couple of videos but there is a few tools that uh give you the engagement rate so that part is all automatized from my side  you use some external external tools to get a specific value of of this engagement rate yeah yeah so when i go so when i go into the person's profile  uh i use a tool that already gives me the engagement rate for the global for the last couple of months, but there is also a human part that needs to be considered.  So, for example, comments are very valuable.  you know, so that part's not considered in any automation.  That part I need to be doing, I need to be looking into by myself.
- [27:30] Lara Vargas:  For example, the the product research, the I have a human resources, so different types of engagement engagements. So, should I focus on all these different types or...  Tell me, what are the different types of engagements? What are you about?  So I will just go through them really quick. So the first one, as I said, is the coaching. Second one, product research and brain.  Storming for marketing teams, creative planning, e-learning and education.  A internal auditing.  A inclusion, assistant for a Teams yes, these are all valid thing well it thinks  absolutely uh legitimate targets absolutely like agile meetings you are more about about what are the audience we want to tackle but yeah i am about how to process any of the all of this into a a result any of this right okay we'll be iterating on on those we will be iterating okay so it brings a lot of value when you are also thinking about what you're going to do not about like not what is the you know like you're making a car but what is the what the what the what the what is the production line right because what you're going to produce but about how you're going to produce because this way we can produce whatever we can We can put whatever piece of text and then get the result of who is actually been posting about it lately.
- [44:06] Dmitriy Grankin:  We need to come up with a set of audiences, process them into a set of channels yeah and comes comes the outreach part perfect  sounds good i will get on to that so when should we have our next talk okay let's exchange on  and create this list yeah yes let's come up with this list and uh i will process it i'll create a document with this with the list of channels okay and you can you can you can do the both ways your own way or you can combine the  approach yes i will i will be doing i will be doing because there's a lot of guys who are  who post on youtube and they are  posting elsewhere as well yeah okay okay sounds good to me I need to finish  finish up so let's continue okay chat okay thank you thanks laura bye-bye bye-bye
- [18:49] Dmitriy Grankin:  You can basically very quickly turn around with different problem areas like this way, because today you can outreach say 20 manual speakers about this thing and set up a few meetings and this just a few meetings would give you a lot of information about if you really want to be do they want you to be what's their culture about that how actively they are looking for something new at all so that The pipeline is that you get these channels, you set up meetings, you get these meetings, and here is the product.  is already there because you had a few meetings with demand expertise in a very precise niche.  So it doesn't matter much about which problem area you are going to export this way right because you can just take a bunch of those randomly and start from those if they are just suddenly it's sound right right okay okay after you got this idea about what's going on, about this contact, about these people, this, uh, then you'd better engage with those creating content with those guys.  And just right after that, you're going to create your own content because you're already well aware of the problem, you have well data, the idea customer profile this way, possibly. And you have experience created content with those guys, you know, what works, what what does not. And After that, you're ready to create your own content.  That's how I feel about this.  Okay.
- [24:22] Dmitriy Grankin:  which is already a lot of information from this meeting possibly if we if we're talking about something that we for sure would understand if there is anything about this at all if we are in the same share share at least some common ground at all if we do share then it's a win if we don't it's already it's also an information a lot of information okay right so we should  really start i should really start reaching out to the people regarding the Project management.  We don't know, because it's just a piece of text that we're putting into the system.  system uh how to set up trello for product management you can you you can put whatever request like how to uh how to make a paper ship right and it will work the same way okay
- [42:40] This chunk occurs towards the end of a conversation between Dmitriy Grankin and Lara Vargas, where they discuss strategies for influencer outreach and marketing. Dmitriy summarizes their plan to identify target audiences and influencers, emphasizing the importance of understanding the audience rather than just the influencers themselves, and outlines the next steps involving YouTube queries and channel processing.
Dmitriy Grankin:  Okay, to wrap up, we want to have meetings with those, we want to create a set of short descriptions about who we are looking for, these short descriptions.  would not be describing the influencers themselves I would be describing their audience okay so a few descriptions of the audience they are uh they are targeting and starting from that from these descriptions you can you can do like uh you can go and search for influencers already but if you use using my tool then the next step is creating a bunch of youtube queries specific queries um these guys are likely to make and then process this curious into a set of uh channels on youtube  yeah so basically


In [60]:
input = count_tokens(input_context)

In [61]:
input* 0.15/1000000

0.0007909499999999999

In [62]:
from core import generic_call,system_msg,user_msg


In [63]:
system_prompt = """
you are helpful assistant that is answering user requests by interpreting  while interpreting the context
You have to rely on the context to answer the user request

User is Dmitry Grankin, Address his as YOU and distinguish them in in the provided context in in the response.

Alsay supply reference to specific meeting with [#meeting_id]
"""




In [64]:

r = await generic_call([system_msg(system_prompt),
                        user_msg(f'context: {input_context}'),
                        user_msg(f'request: {q}')])
r



### Meeting Summary - December 11, 2024

In the meeting, **Dmitriy Grankin** and **Lara Vargas** discussed strategies for identifying and engaging with **micro-influencers** in specific industries. **Dmitriy** emphasized the importance of targeting **domain experts** who have a precise user base, as they can provide valuable insights into potential customer profiles. 

Key action points included:
- Creating a set of short descriptions about the **audiences** they are targeting, rather than the influencers themselves.
- Developing a pipeline for outreach to these influencers to gather insights before creating content.
- Utilizing **YouTube queries** to identify relevant channels and influencers based on specific topics, such as **project management tools** like **Jira** and **Trello**.

**Lara** proposed various use cases for the **VEXA** tool, including its application in **qualitative research** and **training feedback** for coaches. They agreed to compile a list of potential influenc

'### Meeting Summary - December 11, 2024\n\nIn the meeting, **Dmitriy Grankin** and **Lara Vargas** discussed strategies for identifying and engaging with **micro-influencers** in specific industries. **Dmitriy** emphasized the importance of targeting **domain experts** who have a precise user base, as they can provide valuable insights into potential customer profiles. \n\nKey action points included:\n- Creating a set of short descriptions about the **audiences** they are targeting, rather than the influencers themselves.\n- Developing a pipeline for outreach to these influencers to gather insights before creating content.\n- Utilizing **YouTube queries** to identify relevant channels and influencers based on specific topics, such as **project management tools** like **Jira** and **Trello**.\n\n**Lara** proposed various use cases for the **VEXA** tool, including its application in **qualitative research** and **training feedback** for coaches. They agreed to compile a list of potentia

In [48]:
def extract_tagged_content(text: str, tag: str) -> str:
    import re
    # Try exact match first
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # If no exact match, try to find content after opening tag
    pattern = f"<{tag}>(.*?)(?:</?|$)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Example usage:
thinking = extract_tagged_content(r, "thinking")
answer = extract_tagged_content(r, "answer")

In [51]:
display(Markdown(answer))

In the meeting, **Dmitriy Grankin** and **Lara Vargas** discussed strategies for engaging with **micro-influencers** to promote **VEXA**. They emphasized the importance of understanding target audiences and creating content that resonates with them. Key action points included creating short descriptions of target audiences, identifying and reaching out to **micro-influencers**, developing a pipeline for influencer engagement, and analyzing engagement rates. They also planned to create specific YouTube queries to aid in their research and set up meetings with potential influencers to gather insights and foster collaboration.

In [50]:
thinking

'Entities:\n- People: **Dmitriy Grankin**, **Lara Vargas**\n- Companies: None mentioned\n- Products: **VEXA**\n- Ideas: Engaging with **micro-influencers**, influencer outreach strategies, audience targeting, content creation, qualitative research\n- Action Points: \n  - Create a set of short descriptions about target audiences\n  - Identify and outreach to **micro-influencers**\n  - Develop a pipeline for influencer engagement\n  - Create YouTube queries for influencer research\n  - Analyze engagement rates of influencers\n  - Set up meetings with identified influencers'